In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%env TOKENIZERS_PARALLELISM=false

env: TOKENIZERS_PARALLELISM=false


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader

from data import build_dataset, get_collate_fn, get_tokenizer

In [4]:
data = build_dataset()
collate_fn = get_collate_fn()
loader = DataLoader(data, 16, collate_fn=collate_fn, num_workers=8)

loading annotations into memory...
Done (t=0.34s)
creating index...
index created!


In [5]:
batch = next(iter(loader))

In [6]:
images = batch["images"]
input_ids = batch["input_ids"]
attention_mask = batch["attention_mask"]

In [7]:
from model import Img2TextTransformer

In [8]:
model = Img2TextTransformer()

In [9]:
input_ids.shape

torch.Size([16, 18])

In [10]:
model(images, input_ids, attention_mask).shape

torch.Size([16, 18, 50258])

In [11]:
total = 0
for p in model.parameters():
    total += p.numel()
print(total)

32802530


In [12]:
logits = model(images, input_ids[:, :-1], attention_mask[:, :-1]).transpose(-2, -1)

In [13]:
logits.shape

torch.Size([16, 50258, 17])

In [14]:
targets = input_ids[:, 1:]

In [15]:
targets.shape

torch.Size([16, 17])

In [16]:
(F.cross_entropy(logits, targets, reduction="none") * attention_mask[:, 1:]).mean()

tensor(8.0080, grad_fn=<MeanBackward0>)

In [18]:
tokenizer = get_tokenizer()

In [20]:
tokenizer(tokenizer.bos_token)

{'input_ids': [50257], 'attention_mask': [1]}

In [23]:
image = images[0]
input_ids = torch.LongTensor(tokenizer(tokenizer.bos_token)["input_ids"])

In [27]:
model.forward_single(image, input_ids).shape

torch.Size([1, 1, 50258])